In [ ]:
from pymongo import MongoClient
import json

import os
import torch
from torch.utils.data import Dataset, DataLoader
from pymongo import MongoClient
import numpy as np
from scapy.all import Ether, IP
import pymongo
from scapy.all import PcapReader
from scapy.compat import bytes_encode, hex_bytes, bytes_hex
from scapy.layers import l2

import os
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
db_name = "mixed"
collection_name = "test"
output_path = "E:\PCAP_2018\Original\clean_data\IDS_2018_PARTIAL\\test_vector.txt"
label_maps = label_maps = {
        'ftp-bruteforce': 0,
        'ddos-hoic': 1,
        'dos-goldeneye': 2,
        'ddos-loic-http': 3,
        "web_attack_merge": 4,
        'dos-hulk': 5,
        'bot': 6,
        'ssh-bruteforce': 7,
        'dos-slowhttptest': 8,
        'dos-slowloris': 9,
        'benign': 10,
        'ddos-loic-udp': 11,
        'inflitration': 12
}
client = MongoClient('localhost', 27017)
db = client[db_name]
collection = db[collection_name]
img_size = 16
channels = 20

In [ ]:
def str_to_pkt(string):
    return l2.Ether(hex_bytes(string))

def reset_addr(pkt):
    assert pkt.haslayer('Ether') and pkt.haslayer('IP')
    pkt.getlayer('Ether').dst = "00:00:00:00:00:00"
    pkt.getlayer('Ether').src = "00:00:00:00:00:00"
    pkt.getlayer('IP').src = "0.0.0.0"
    pkt.getlayer('IP').dst = "0.0.0.0"
    return pkt
def pkt_to_sparse_format(pkts_list, img_size=16, channels=20):
    assert isinstance(pkts_list, list)
    idx1, idx2, vals = [], [], []
    for i, ele in enumerate(pkts_list):
        if i >= channels:
            break
        for j, b in enumerate(bytes_encode(reset_addr(str_to_pkt(ele)))):
            if j >= img_size * img_size:
                break
            pixel = int(b)
            if pixel > 0:
                idx1.append(i)
                idx2.append(j)
                vals.append(pixel)
    return idx1, idx2, vals

In [ ]:
with open(output_path, 'w') as file:
    for bson_ds in collection.find():
        bson_ds['label'] = bson_ds['label'].lower()
        label = label_maps[bson_ds['label']]
        last_time = bson_ds['last_seen_time'] - bson_ds['begin_time']
        byte_len = bson_ds['byte_len']
        idx1, idx2, values = pkt_to_sparse_format(bson_ds['pkts_list'])

        i = torch.LongTensor([idx1, idx2])
        v = torch.FloatTensor(values)
        shape = torch.Size([channels, img_size * img_size])
        sparse_matrix = torch.sparse.FloatTensor(i, v, shape).to_dense()
        #sparse_matrix = torch.nn.functional.one_hot(sparse_matrix.long(), num_classes=256)
        data_str = json.dumps({
                "sparse_matrix": sparse_matrix.tolist(),
                "label": label
            })
        file.write(data_str + '\n')
        file.flush()

In [ ]:
db_name = "mixed_test"
collection_name = "test"
output_path = "E:\PCAP_2018\Original\clean_data\IDS_2018_PARTIAL\\test_vector.txt"
label_maps = label_maps = {
        'ftp-bruteforce': 0,
        'ddos-hoic': 1,
        'dos-goldeneye': 2,
        'ddos-loic-http': 3,
        "web_attack_merge": 4,
        'dos-hulk': 5,
        'bot': 6,
        'ssh-bruteforce': 7,
        'dos-slowhttptest': 8,
        'dos-slowloris': 9,
        'benign': 10,
        'ddos-loic-udp': 11,
        'inflitration': 12
}
client = MongoClient('localhost', 27017)
db = client[db_name]
collection = db[collection_name]
img_size = 16
channels = 20

In [ ]:
with open(output_path, 'w') as file:
    for bson_ds in tqdm(collection.find()):

        bson_ds['label'] = bson_ds['label'].lower()
        label = label_maps[bson_ds['label']]
        last_time = bson_ds['last_seen_time'] - bson_ds['begin_time']
        byte_len = bson_ds['byte_len']
        idx1, idx2, values = pkt_to_sparse_format(bson_ds['pkts_list'])

        i = torch.LongTensor([idx1, idx2])
        v = torch.FloatTensor(values)
        shape = torch.Size([channels, img_size * img_size])
        sparse_matrix = torch.sparse.FloatTensor(i, v, shape).to_dense()
        #sparse_matrix = torch.nn.functional.one_hot(sparse_matrix.long(), num_classes=256)
        data_str = json.dumps({
                "sparse_matrix": sparse_matrix.tolist(),  # Convert tensor to list for JSON serialization
                "label": label
            })
        file.write(data_str + '\n')